# Bonus Unité 1: Finetuner un modèle pour faire de l'appel de fonctions

Dans ce tutoriel, **nous allons finetuner un LLM pour pouvoir faire de l'appel de fonctions.**

Ce notebook fait parti du cours <a href="https://huggingface.co/learn/agents-course/fr">sur les agents d'Hugging Face</a>, un cours gratuit qui vous guidera, du **niveau débutant à expert**, pour comprendre, utiliser et construire des agents.


<img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png" alt="Agent Course"/>


## Prérequis de l'exercice 🏗️

Avant de vous plonger dans le *notebook*, vous devez :

🔲 📚 **Etudier la section [Qu'est-ce que l'appel de fonctions ?](https://huggingface.co/learn/agents-course/fr/bonus-unit1/what-is-function-calling)**

🔲 📚 **Etudier la section [Finetunons un modèle pour pouvoir faire de l'appel de fonctions](https://huggingface.co/learn/agents-course/fr/bonus-unit1/fine-tuning)**

# Étape 0 : Demande d'accès à Gemma sur Hugging Face

<img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit1/gemma.png" alt="Gemma"/>


Pour accéder à Gemma sur Hugging Face :

1. **Assurez-vous d'être connecté** à votre compte Hugging Face.

2. Allez sur https://huggingface.co/google/gemma-2-2b-it

3. Cliquez sur **Acknowledge license** et remplissez le formulaire.

Vous pouvez également utiliser un autre modèle et modifier le code en conséquence (cela peut être un bon exercice pour vous assurer que vous savez comment finetuner sur la tâche d'appel de fonction).

Vous pouvez par exemple utiliser :

- [HuggingFaceTB/SmolLM2-1.7B-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-1.7B-Instruct)

- [meta-llama/Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)

## Étape 1 : Configurer le GPU 💪

Si vous êtes sur Colab :

- Pour **accélérer l'entraînement du finetuning, nous allons utiliser un GPU**. Pour cela, allez dans `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1"/>

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2"/>


### Important

Pour cette unité, **avec le niveau gratuit de Colab** il faudra environ **6h pour entraîner**.

Trois solutions s'offrent à vous pour accélérer le processus :

1. Entraînez-vous sur votre ordinateur si vous avez des GPU. Cela peut prendre du temps mais il y a moins de risques de dépassement de temps.

2. Utiliser un Google Colab Pro qui permet d'utiliser un GPU A100 (alors plus que 15-20min d'entraînement).

3. Suivez le code pour apprendre à comment faire mais sans pouvoir entraîner.

## Étape 2 : Installer les dépendances 📚

Nous avons besoin de plusieurs bibliothèques :

- `bitsandbytes` pour la quantification
- `peft` pour les adaptateurs LoRA
- `transformers` pour le chargement du modèle
- `datasets` pour le chargement et l'utilisation du jeu de données de finetuning
- `trl` pour la classe Trainer

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U tensorboardX
!pip install -q wandb
!pip install -q -U torchvision
!pip install -q -U transformers

## Etape 3 : Créez votre *token* Hugging Face pour pousser votre modèle sur le Hub

Pour pouvoir partager votre modèle avec la communauté, il y a encore quelques étapes à suivre :

1️⃣ (Si ce n'est pas déjà fait) créez un compte sur HF ➡ https://huggingface.co/join

2️⃣ Connectez-vous et ensuite, vous devez stocker votre *token* d'authentification du site Hugging Face.

- Créez un nouveau *token* (https://huggingface.co/settings/tokens) **avec un rôle d'écriture**

<img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit1/create_write_token.png" alt="Create HF Token" width="50%">

3️⃣ Stockez votre *token* dans une variable d'environnement sous le nom « HF_TOKEN »
- **Faites très attention à ne pas le partager avec d'autres personnes** !

## Étape 4 : Importer les bibliothèques

N'oubliez pas de mettre votre *token* HF.

In [1]:
from enum import Enum
from functools import partial
import pandas as pd
import torch
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, set_seed
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, TaskType

seed = 42
set_seed(seed)

import os

# Mettez votre token HF ici
os.environ['HF_TOKEN']="hf_xxxxxxx" # le token doit avoir un droit d'accès d'écriture

## Étape 5 : Traitement du jeu de données

Afin d'entraîner le modèle, nous devons **formater les entrées en fonction de ce que nous voulons que le modèle apprenne**.

Pour ce tutoriel, j'ai amélioré un jeu de données populaire pour l'appel de fonction "NousResearch/hermes-function-calling-v1" en ajoutant des étapes de **raisonnement** issues de **deepseek-ai/DeepSeek-R1-Distill-Qwen-32B**.

Mais pour que le modèle puisse apprendre, nous devons **formater la conversation correctement**. Si vous avez suivi l'Unité 1, vous savez que le passage d'une liste de messages à un *prompt* est géré par le **chat_template**. Le chat_template par défaut de gemma-2-2B ne contient pas d'appels d'outils. Nous allons donc devoir le modifier !

C'est le rôle de notre fonction **preprocess**. Passer d'une liste de messages à un *prompt* que le modèle peut comprendre.

In [2]:
model_name = "google/gemma-2-2b-it"
dataset_name = "Jofthomas/hermes-function-calling-thinking-V1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"


def preprocess(sample):
      messages = sample["messages"]
      first_message = messages[0]

      # Au lieu d'ajouter un prompt système, nous fusionnons le contenu dans le premier message de l'utilisateur.
      if first_message["role"] == "system":
          system_message_content = first_message["content"]
          # Fusionner le contenu du système avec le premier message de l'utilisateur
          messages[1]["content"] = system_message_content + "Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>\n\n" + messages[1]["content"]
          # Supprimer le message système de la conversation
          messages.pop(0)

      return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}



dataset = load_dataset(dataset_name)
dataset = dataset.rename_column("conversations", "messages")

## Étape 6 : Un jeu de données dédié à cette Unité

Pour cette Unité bonus, nous avons créé un jeu de données personnalisé basé sur [NousResearch/hermes-function-calling-v1](https://huggingface.co/datasets/NousResearch/hermes-function-calling-v1), qui est considéré comme une **référence** en matière de jeu de données pour l'appel de fonctions.

Bien que le jeu de données original soit excellent, il ne **comprend pas** d'étape de **réflexion**.

Pour de l'appel de fonction, une telle étape est optionnelle, mais des travaux récents - comme le modèle **deepseek** ou le papier [*Test-Time Compute*](https://huggingface.co/papers/2408.03314) - suggèrent que donner à un LLM le temps de « penser » avant de répondre (ou dans ce cas, **avant** d'entreprendre une action) peut **améliorer de manière significative** la performance du modèle.

J'ai donc décidé de créer un sous-ensemble de ce jeu de données et de le donner à [deepseek-ai/DeepSeek-R1-Distill-Qwen-32B](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B) afin de créer quelques *tokens* de réflexion `<think>` avant tout appel de fonction. Ce qui a abouti au jeu de données suivant :
![Input Dataset](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit1/dataset_function_call.png)

In [3]:
dataset = dataset.map(preprocess, remove_columns="messages")
dataset = dataset["train"].train_test_split(0.1)
print(dataset)

dataset["train"] = dataset["train"].select(range(100))
dataset["test"] = dataset["test"].select(range(10))

Map:   0%|          | 0/3570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3213
    })
    test: Dataset({
        features: ['text'],
        num_rows: 357
    })
})


## Étape 7 : Vérification des entrées

Regardons manuellement à quoi ressemble une entrée !

Dans cet exemple, nous avons :

1. Un *Message de l'utilisateur* contenant les **informations nécessaires avec la liste des outils disponibles** entre `<tools></tools>` puis la requête de l'utilisateur, ici :  `"Can you get me the latest news headlines for the United States?"` (`« Pouvez-vous me donner les derniers titres de l'actualité pour les Etats-Unis ? »`)

2. Un *message assistant* appelé ici « modèle » pour répondre aux critères des modèles gemma contenant deux nouvelles phases, une phase **« penser »** contenue dans `<think></think>` et une phase **« agir »** contenue dans `<tool_call></tool_call>`.

3. Si le modèle contient un `<tools_call>`, nous ajouterons le résultat de cette action dans un nouveau message **« Tool »** contenant un `<tool_response></tool_response>` avec la réponse de l'outil.

In [4]:
# Voyons comment nous avons formaté le jeu de données
print(dataset["train"][8]["text"])

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_news_headlines', 'description': 'Get the latest news headlines', 'parameters': {'type': 'object', 'properties': {'country': {'type': 'string', 'description': 'The country for which headlines are needed'}}, 'required': ['country']}}}, {'type': 'function', 'function': {'name': 'search_recipes', 'description': 'Search for recipes based on ingredients', 'parameters': {'type': 'object', 'properties': {'ingredients': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The list of ingredients'}}, 'required': ['ingredients']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall

In [5]:
# Contrôle 
print(tokenizer.pad_token)
print(tokenizer.eos_token)

<pad>
<eos>


## Étape 8 : Modifions le *tokenizer*

En effet, comme nous l'avons vu dans l'Unité 1, le *tokenizer* divise le texte en sous-mots par défaut. Ce n'est **pas** ce que nous voulons pour nos nouveaux *tokens* spéciaux !

Bien que nous ayons segmenté notre exemple en utilisant `<think>`, `<tool_call>`, et `<tool_response>`, le *tokenizer* ne les traite **pas** encore comme des *tokens* entiers - il essaie toujours de les décomposer en plus petits morceaux. Pour s'assurer que le modèle interprète correctement notre nouveau format, nous devons **ajouter ces *tokens*** à notre *tokenizer*.

De plus, puisque nous avons changé le `chat_template` dans notre fonction **preprocess** pour formater les conversations comme des messages dans un *prompt*, nous devons aussi modifier le `chat_template` dans le *tokenizer* pour refléter ces changements.

In [6]:
class ChatmlSpecialTokens(str, Enum):
    tools = "<tools>"
    eotools = "</tools>"
    think = "<think>"
    eothink = "</think>"
    tool_call="<tool_call>"
    eotool_call="</tool_call>"
    tool_response="<tool_response>"
    eotool_response="</tool_response>"
    pad_token = "<pad>"
    eos_token = "<eos>"
    @classmethod
    def list(cls):
        return [c.value for c in cls]

tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list()
    )
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             attn_implementation='eager',
                                             device_map="auto")
model.resize_token_embeddings(len(tokenizer))
model.to(torch.bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256008, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps

## Étape 9 : Configurons notre LoRA

Nous allons définir les paramètres de notre adaptateur. Ce sont les paramètres les plus importants du LoRA car ils définissent la taille et l'importance des adaptateurs que nous entraînons.

In [7]:
from peft import LoraConfig

# TODO : Configurer les paramètres du LoRA
# r : dimension du rang des matrices de mise à jour du LoRA (plus petit = plus de compression)
rank_dimension = 16
# lora_alpha: facteur d'échelle pour les couches LoRA (plus élevé = adaptation plus forte)
lora_alpha = 64
# lora_dropout: probabilité du dropout pour les couches LoRA (aide à prévenir le surentraînement)
lora_dropout = 0.05

peft_config = LoraConfig(r=rank_dimension,
                         lora_alpha=lora_alpha,
                         lora_dropout=lora_dropout,
                         target_modules=["gate_proj","q_proj","lm_head","o_proj","k_proj","embed_tokens","down_proj","up_proj","v_proj"], # quelle couche du transformer devons-nous cibler ?
                         task_type=TaskType.CAUSAL_LM)

## Étape 10 : Définissons le Trainer et les hyperparamètres du finetuning

Dans cette étape, nous définissons le Trainer, la classe que nous utilisons pour finetuner notre modèle et les hyperparamètres.

In [8]:
username="Jofthomas"# REMPLACER par votre nom d'utilisateur Hugging Face
output_dir = "gemma-2-2B-it-thinking-function_calling-V0" # Le répertoire où les checkpoints du modèle entraîné, les logs et les autres artefacts seront sauvegardés. Il sera également le nom par défaut du modèle lorsqu'il sera poussé vers le hub s'il n'est pas redéfini ultérieurement
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 4
logging_steps = 5
learning_rate = 1e-4 # Le taux d'apprentissage initial de l'optimiseur

max_grad_norm = 1.0
num_train_epochs=1
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 1500

training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    eval_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    bf16=True,
    hub_private_repo=False,
    push_to_hub=False,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    packing=True,
    max_seq_length=max_seq_length,
)

Comme Trainer, nous utilisons le `SFTTrainer` (Supervised Fine-Tuning Trainer) pour du finetuning supervisé.

In [9]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    peft_config=peft_config,
)

/home/user/miniconda/lib/python3.9/site-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


Applying chat template to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Ici, nous lançons l'entraînement 🔥. C'est le moment idéal pour faire une pause et prendre un café ☕.

In [10]:
trainer.train()
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,1.236800,1.240833


/home/user/miniconda/lib/python3.9/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


## Etape 11 : Poussons le modèle et le *tokenizer* sur le Hub

Poussons notre modèle et notre *tokenizer* sur le Hub ! Le modèle sera poussé sous votre nom d'utilisateur + le répertoire de sortie que nous avons spécifié plus tôt.

In [11]:
trainer.push_to_hub(f"{username}/{output_dir}")

events.out.tfevents.1739887545.r-jofthomas-fttest-kff5bkw4-24c03-yhiku:   0%|          | 0.00/6.88k [00:00<?, …

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.48G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Jofthomas/gemma-2-2B-it-thinking-function_calling-V0/commit/74db7b9fd8e5c6591db5851db70069bf3016fe50', commit_message='Jofthomas/gemma-2-2B-it-thinking-function_calling-V0', commit_description='', oid='74db7b9fd8e5c6591db5851db70069bf3016fe50', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jofthomas/gemma-2-2B-it-thinking-function_calling-V0', endpoint='https://huggingface.co', repo_type='model', repo_id='Jofthomas/gemma-2-2B-it-thinking-function_calling-V0'), pr_revision=None, pr_num=None)

Puisque nous avons également modifié le **chat_template** qui est contenu dans le *tokenizer*, poussons également le tokenizer avec le modèle.

In [12]:
tokenizer.eos_token = "<eos>"
# pousser le tokenizer sur le hub (remplacer par votre nom d'utilisateur et votre nom d'utilisateur spécifié précédemment)
tokenizer.push_to_hub(f"{username}/{output_dir}", token=True)

README.md:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jofthomas/gemma-2-2B-it-thinking-function_calling-V0/commit/5eae94697866df5f9f52bc7c7be677f100a7f339', commit_message='Upload tokenizer', commit_description='', oid='5eae94697866df5f9f52bc7c7be677f100a7f339', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jofthomas/gemma-2-2B-it-thinking-function_calling-V0', endpoint='https://huggingface.co', repo_type='model', repo_id='Jofthomas/gemma-2-2B-it-thinking-function_calling-V0'), pr_revision=None, pr_num=None)

## Étape 12 : Testons maintenant notre modèle !

Pour cela, nous allons :

1. Charger l'adaptateur  partir du Hub !
2. Charger le modèle de base : **« google/gemma-2-2b-it »** depuis le Hub
3. Redimensionner le modèle avec les nouveaux *tokens* que nous avons introduits !

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch

bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )

peft_model_id = f"{username}/{output_dir}" # à remplacer par votre adaptateur entraîné
device = "auto"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             device_map=device,
                                             )
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, peft_model_id)
model.to(torch.bfloat16)
model.eval()

adapter_config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.48G [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(256006, 2304, padding_idx=0)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.05, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.BFloat16Tensor of size 16x256006 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.BFloat16Tensor of size 2304x16 (cuda:0)])
          (lora_magnitude_vector): ModuleDict()
        )
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
          

In [20]:
print(dataset["test"][8]["text"])

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert from one currency to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to convert'}, 'from_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'to_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['amount', 'from_currency', 'to_currency']}}}, {'type': 'function', 'function': {'name': 'calculate_distance', 'description': 'Calculate the distance between two locations', 'parameters': {'type': 'object', 'properties': {'start_location': {'type': 'string', 'description': 'The star

### Test du modèle 🚀

Dans ce cas, nous prendrons le début de l'un des échantillons du jeu de test et nous espérons qu'il générera la sortie attendue.

Puisque nous voulons tester les capacités d'appel de fonctions de notre modèle finetuné, l'entrée sera un message d'utilisateur avec les outils disponibles, un message de l'utilisateur et un message de l'utilisateur.


### Avertissement ⚠️

Le jeu de données que nous utilisons **ne contient pas suffisamment de données d'entraînement** et est purement **à des fins éducatives**. Par conséquent, **les résultats de votre modèle entraîné peuvent différer** des exemples montrés dans ce cours. **Ne vous découragez pas** si vos résultats varient - notre objectif principal est d'illustrer les concepts de base plutôt que de produire un modèle entièrement optimisé ou prêt pour la production.

In [21]:
# Ce prompt est un sous-échantillon de l'un des exemples du jeu de test. Dans cet exemple, nous démarrons la génération après le début de la génération du modèle.
prompt="""<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert from one currency to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to convert'}, 'from_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'to_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['amount', 'from_currency', 'to_currency']}}}, {'type': 'function', 'function': {'name': 'calculate_distance', 'description': 'Calculate the distance between two locations', 'parameters': {'type': 'object', 'properties': {'start_location': {'type': 'string', 'description': 'The starting location'}, 'end_location': {'type': 'string', 'description': 'The ending location'}}, 'required': ['start_location', 'end_location']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call>Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>

Hi, I need to convert 500 USD to Euros. Can you help me with that?<end_of_turn><eos>
<start_of_turn>model
<think>"""

inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
inputs = {k: v.to("cuda") for k,v in inputs.items()}
outputs = model.generate(**inputs,
                         max_new_tokens=300,# A adapter si nécessaire
                         do_sample=True,
                         top_p=0.95,
                         temperature=0.01,
                         repetition_penalty=1.0,
                         eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert from one currency to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to convert'}, 'from_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'to_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['amount', 'from_currency', 'to_currency']}}}, {'type': 'function', 'function': {'name': 'calculate_distance', 'description': 'Calculate the distance between two locations', 'parameters': {'type': 'object', 'properties': {'start_location': {'type': 'string', 'description': 'The star

## Félicitations
Félicitations pour avoir terminé cette première unité bonus 🥳

Vous venez de **voir ce qu'est l'appel de fonction et comment finetuner votre modèle pour faire de l'appel de fonction** !

Si c'est la première fois que vous faites cela, il est normal que vous vous sentiez dérouté. Prenez le temps de consulter la documentation et de comprendre chaque partie du code et pourquoi nous l'avons fait de cette façon.

N'hésitez pas non plus à essayer **de finetuner différents modèles**. La **meilleure façon d'apprendre est d'essayer**.

### Continuez à apprendre, restez géniaux 🤗